# Assignment 2

In [3]:
import numpy as np
import scipy as sp
import matplotlib as mp
import matplotlib.pyplot as plt

from scipy.integrate import quad, dblquad, trapz
from textwrap import wrap

%matplotlib inline
%config InlineBackend.figure_format = 'pdf'

## Question 5

### (a)

We have a loop of radius $10$cm lying in the xy-plane with current of $I$ flowing.

To calculate the magnetic field produced by this loop we use the Biot-Savart Law:

$$\vec{B}(\vec{r}) = \frac{\mu_0 I}{4 \pi} \int \frac{\text{d}l \times \hat{r'}}{r'^2}$$

$$\text{where} \left\{ \begin{array}{ll}
            \vec{r}  & \text{is the position vector of the point of evaluation} \\
            \vec{r'} & \text{is the displacement vector from a point in the current loop to the point of evaluation} \\
            l        & \text{is the length of the current loop} \\
            \end{array} \right.$$      
            

**Let:**

>Radius of the current loop: $R$

>Positions of elements on the loop: $\vec{R}$

>Angle from the center of the current loop to an element: $\theta$ 

>Line segment of the current loop: $l$

>Locations in space: $\vec{r} = (x, y, z)$

**Then:**
$$\vec{R} = [R\cos(\theta),\ y-R\sin(\theta), \ z]$$

$$\text{d}l = R \ \text{d} \theta \ \hat{\theta} = [-R \sin(\theta), \ R \cos(\theta), \ 0] \ \text{d}\theta$$

$$\vec{r'} = \vec{r} - \vec{R} = [x-R\cos(\theta), \ y-R\sin(\theta), \ z]$$

**Taking the cross product:**
\begin{align}
\text{d}l \times \vec{r'} &= zR\cos(\theta) \ \hat{x} + zR\sin(\theta) \ \hat{y} + [-R\sin(\theta)(y-R\sin(\theta)) - R\cos(\theta)(x-R\cos(\theta))] \ \hat{z} \ \text{d}\theta\\
&= zR\cos(\theta) \ \hat{x} + zR\sin(\theta) \ \hat{y} + [-yR\sin(\theta)-xR\cos(\theta) + R^2] \hat{z} \ \text{d}\theta 
\end{align}

**The term inside the intergral:**
\begin{align}
\frac{\text{d}l \times \vec{r'}}{r'^2} &= \frac{zR\cos(\theta) \ \hat{x} + zR\sin(\theta) \ \hat{y} + [-yR\sin(\theta)-xR\cos(\theta) + R^2] \hat{z}}{\left\{[x-R\cos(\theta)]^2 + [y-R\sin(\theta)]^2 + z^2 \right\}^\frac{3}{2}}
\end{align}

**The magnetic field:**
$$\vec{B}(\vec{r}) = \frac{\mu_0 I}{4 \pi} \int \text{d}\theta \ \frac{zR\cos(\theta) \ \hat{x} + zR\sin(\theta) \ \hat{y} + [-yR\sin(\theta)-xR\cos(\theta) + R^2] \hat{z}}{\left\{[x-R\cos(\theta)]^2 + [y-R\sin(\theta)]^2 + z^2 \right\}^\frac{3}{2} }$$

**Make it dimensionless:**
$$\vec{B}(\vec{r}) \frac{4 \pi}{\mu_0 I}  = \int \text{d}\theta \ \frac{\frac{z}{R}\cos(\theta) \ \hat{x} + \frac{z}{R}\sin(\theta) \ \hat{y} - \left[\frac{y}{R}\sin(\theta) + \frac{x}{R}\cos(\theta) - 1\right] \hat{z}}{\left\{ \left[\frac{x}{R}-\cos(\theta)\right]^2 + \left[\frac{y}{R}-\sin(\theta)\right]^2 + \left(\frac{z}{R}\right)^2 \right\}^\frac{3}{2} }$$

**Turn the magnetic field into code:**

In [4]:
# Magnetic fields in x
def integrand_x(x, y, z, theta, R=1):
    r = ((x/R - np.cos(theta))**2 + (y/R - np.sin(theta))**2 + (z/R)**2)**.5
    Bx = (z/R)*np.cos(theta) / r**3
    return Bx

@np.vectorize
def Bx(x, y, z, R=1):
    B_x = quad(lambda theta: integrand_x(x, y, z, theta, R), 0, 2*np.pi)
    return B_x[0]

# Magnetic fields in y
def integrand_y(x, y, z, theta, R=1):
    r = ((x/R - np.cos(theta))**2 + (y/R - np.sin(theta))**2 + (z/R)**2)**.5
    By = (z/R)*np.sin(theta) / r**3
    return By

@np.vectorize
def By(x, y, z, R=1):
    B_y = quad(lambda theta: integrand_y(x, y, z, theta, R), 0, 2*np.pi)
    return B_y[0]

# Magnetic fields in z
def integrand_z(x, y, z, theta, R=1):
    r = ((x/R - np.cos(theta))**2 + (y/R - np.sin(theta))**2 + (z/R)**2)**.5
    Bz = -( (y/R)*np.sin(theta) + (x/R)*np.cos(theta) - 1) / r**3
    return Bz

@np.vectorize
def Bz(x, y, z, R=1):
    B_z = quad(lambda theta: integrand_z(x, y, z, theta, R), 0, 2*np.pi)
    return B_z[0]

In [31]:
# Create a meshgrid for the space of integration
# ==============================================

# Dimesions and resolution for the meshgrid 
r_max = 2
r_min = -2
res = 20

x = np.linspace(r_min, r_max, res)
y = 0 # np.linspace(r_min, r_max, res)
z = np.linspace(r_min, r_max, res)

GX, GZ = np.meshgrid(x, z)
GY = y

In [32]:
%%time
# Find the magnetic fields
# ========================

oldB_x = Bx(GX, GY, GZ, 1)
oldB_y = By(GX, GY, GZ, 1)
oldB_z = Bz(GX, GY, GZ, 1)

CPU times: user 1.77 s, sys: 20.8 ms, total: 1.79 s
Wall time: 1.81 s


In [57]:
r_max = 2
r_min = -2
res = 20

x = np.linspace(r_min, r_max, res)
y = 0 # np.linspace(r_min, r_max, res)
z = np.linspace(r_min, r_max, res)
theta, dtheta = np.linspace(0, 2*np.pi, 20, retstep=True)

GX, GZ, GT = np.meshgrid(x, z, theta)
GY = y

In [112]:
%%time
dB_x = integrand_x(GX, GY, GZ, GT)
B_x = trapz(dB_x, dx=dtheta, axis=2)

dB_z = integrand_z(GX, GY, GZ, GT)
B_z = trapz(dB_z, dx=dtheta, axis=2)

dB_y = integrand_y(GX, GY, GZ, GT)
B_y = trapz(dB_y, dx=dtheta, axis=2)

def By(x, y, z, theta, dtheta):
    return trapz(integrand_y(x, y, z, theta), dx=dtheta,axis=-1)
def Bz(x, y, z, theta, dtheta):
    return trapz(integrand_z(x, y, z, theta), dx=dtheta,axis=-1)

CPU times: user 6.71 ms, sys: 2.34 ms, total: 9.06 ms
Wall time: 7.17 ms


In [59]:
# Plot it out
# ===========

fig, ax = plt.subplots(figsize=(7, 5))

norm = (B_x**2 + B_z**2)**0.5

Graph = ax.pcolormesh(x, z, norm, cmap='inferno_r', vmax=25)
#ax.streamplot(x, z, B_x, B_z, linewidth=0.75,
#             density=1.5, arrowstyle='->', arrowsize=1)

ax.quiver(x, z, B_x/norm, B_z/norm, color='xkcd:green',linewidth=0.75, pivot='mid')
# Graph = ax.imshow(E[2], extent=(-2, 2, -2, 2), vmax=100, cmap='inferno_r')

ax.set_xlim(-2, 2)
ax.set_ylim(-2, 2)
ax.set_xlabel(r'Distance ($\hat{x}$) [dm]', fontsize=12)
ax.set_ylabel(r'Distance ($\hat{z}$) [dm]', fontsize=12)
ax.set_title('Magnetic field of a current loop in the xz-plane', fontsize=14)

cbar = fig.colorbar(Graph, shrink=1, aspect=20)
cbar.ax.set_ylabel(r'$\left| \vec{B} \right|$ $\left[\frac{4 \pi}{\mu_0 I}\right]$', 
                  rotation=0, fontsize=16, labelpad=40)

caption = r"Magnetic field of a current loop located in the xy-plane traveling counterclockwise. The heat map of field magnitude has a max cutoff value at $25 \frac{4 \pi}{\mu_0 I}$ for display purposes, but higher values exist. "
fig.text(0.05, -0.15, "\n".join(wrap(caption, 75)), ha='left', fontsize=12, wrap=False)

plt.show()

<Figure size 504x360 with 2 Axes>

In [60]:
fig, ax = plt.subplots(figsize=(7, 5))

norm = ((oldB_x-B_x)**2 + (oldB_z-B_z)**2)**0.5

Graph = ax.pcolormesh(x, z, norm, cmap='inferno_r')
#ax.streamplot(x, z, B_x, B_z, linewidth=0.75,
#             density=1.5, arrowstyle='->', arrowsize=1)

ax.quiver(x, z, (oldB_x-B_x)/norm, (oldB_z-B_z)/norm, color='xkcd:green',linewidth=0.75, pivot='mid')
# Graph = ax.imshow(E[2], extent=(-2, 2, -2, 2), vmax=100, cmap='inferno_r')

ax.set_xlim(-2, 2)
ax.set_ylim(-2, 2)
ax.set_xlabel(r'Distance ($\hat{x}$) [dm]', fontsize=12)
ax.set_ylabel(r'Distance ($\hat{z}$) [dm]', fontsize=12)
ax.set_title('Magnetic field of a current loop in the xz-plane', fontsize=14)

cbar = fig.colorbar(Graph, shrink=1, aspect=20)
cbar.ax.set_ylabel(r'$\left| \vec{B} \right|$ $\left[\frac{4 \pi}{\mu_0 I}\right]$', 
                  rotation=0, fontsize=16, labelpad=40)

caption = r"Magnetic field of a current loop located in the xy-plane traveling counterclockwise. The heat map of field magnitude has a max cutoff value at $25 \frac{4 \pi}{\mu_0 I}$ for display purposes, but higher values exist. "
fig.text(0.05, -0.15, "\n".join(wrap(caption, 75)), ha='left', fontsize=12, wrap=False)

plt.show()

/Users/kev/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in true_divide
  if __name__ == '__main__':


<Figure size 504x360 with 2 Axes>

\newpage 

### (b)
Please see attached hand written page for diagram

Here we have a small rotating loop of radius $5$cm located $12$cm away from the original loop on the $\hat{z}$ axis. 

**Let:**

> Radius of the small loop: $s$

> Position of elements in the small loop: $\vec{S}$

> Distance of the small loop from the original loop: $z_0 \ \hat{z}$

> Rate at which the small loop is rotating: $\omega t \ \hat{x}$

> Radial unit vector in the small loop: $\hat{s}$

> Azimuthal angle between the $x$-axis and $\hat{s}$: $\phi$

> Normal vector to the surface enclosed by the small current loop: $\hat{n}$

> Polar angle between $\hat{n}$ and the $z$-axis: $\varphi$

> Displacement vector from an element in the original loop to an element in the small loop: $\vec{r''}$

From inspecting the diagram, we can see that: 

> $\hat{s} = \cos{\phi}\hat{x} + \sin{\phi}\sin{\varphi} \hat{y} + \sin{\phi}\cos{\varphi} \hat{z}$

> $\hat{n} = \sin{\varphi}\hat{y} + \cos{\varphi}\hat{z}$

> $\vec{S} = S\cos{\phi}\hat{x} + S\sin{\phi}\sin{\varphi} \hat{y} + [S\sin{\phi}\cos{\varphi} + z_0] \hat{z}$

> $\vec{r''} = \vec{S} - \vec{R} = [S\cos{\phi} - R\cos{\theta}, \ S\sin{\phi}\sin{\varphi} - R\sin{\theta},\ S\sin{\phi}\cos{\varphi} + z_0]$

Since the loop is rotating along the $\hat{x}$, we can conclude:

> $\varphi = \omega t$

**Current induced in the loop:**

\begin{align}
I &= \int_0^{2\pi} \int_0^s \vec{B} \cdot \text{d} \vec{a} \\
  &= \int_0^{2\pi} \int_0^s \vec{B} \cdot \hat{n} \ s \ \text{d}s \ \text{d} \phi
\end{align}

Code the current:

In [84]:
@np.vectorize
def B_norm(S, phi, Vphi, z_0=1.2):
#     B_x = Bx(S*np.cos(phi), S*np.sin(phi)*np.cos(Vphi), S*np.sin(phi)*np.cos(Vphi)+z_0)
    x = S*np.cos(phi)
    y = S*np.sin(phi)*np.cos(Vphi)
    z = S*np.sin(phi)*np.cos(Vphi) + z_0
    
    B = np.array([0, By(x, y, z), Bz(x, y, z)])
    n = np.array([0, np.sin(Vphi), np.cos(Vphi)])
    
    Bn = np.dot(B, n)
    return Bn

@np.vectorize
def current(R=1, s=0.5, z_0=1.2, phi=0.1*np.pi):
    I = dblquad(lambda S, phi: B_norm(S, phi, Vphi)*S, 0, s, lambda phi: 0, lambda phi: 2*np.pi)
    return I

In [134]:
@np.vectorize
def B_norm(S, phi, Vphi, z_0=1.2):
#     B_x = Bx(S*np.cos(phi), S*np.sin(phi)*np.cos(Vphi), S*np.sin(phi)*np.cos(Vphi)+z_0)
    theta, dtheta = np.linspace(0, 2*np.pi, 20, retstep=True)

    x = S*np.cos(phi)
    y = S*np.sin(phi)*np.cos(Vphi)
    z = S*np.sin(phi)*np.cos(Vphi) + z_0
    
    B_y = By(x, y, z, theta, dtheta)
    B_x = np.zeros_like(B_y)
    B = np.array([B_x, B_y, Bz(x, y, z, theta, dtheta)])
    n = np.array([0, np.sin(Vphi), np.cos(Vphi)])
    
    Bn = np.dot(B, n)
    return Bn

@np.vectorize
def current(R=1, s=0.5, z_0=1.2, Vphi=np.pi/2):
    S = np.linspace(0, s, 20)
    Phi = np.linspace(0, 2*np.pi, 20)
    SS, PPhi = np.meshgrid(S, Phi)
    B = B_norm(SS, PPhi, Vphi)
    I = trapz(trapz(B*SS, x=S, axis=0), x=Phi, axis=0)
    return I

In [155]:
def B_norm(S, phi, Vphi, z_0=1.2):
#     B_x = Bx(S*np.cos(phi), S*np.sin(phi)*np.cos(Vphi), S*np.sin(phi)*np.cos(Vphi)+z_0)
    theta, dtheta = np.linspace(0, 2*np.pi, 20, retstep=True)
    S, phi, theta = np.meshgrid(S, phi, theta)

    x = S*np.cos(phi)
    y = S*np.sin(phi)*np.cos(Vphi)
    z = S*np.sin(phi)*np.cos(Vphi) + z_0
    
    B_y = By(x, y, z, theta, dtheta)
    B_x = np.zeros_like(B_y)
    B_z = Bz(x, y, z, theta, dtheta)
    Bn = B_y*np.sin(Vphi)+B_z*np.cos(Vphi)
    return Bn

@np.vectorize
def current(R=1, s=0.5, z_0=1.2, Vphi=np.pi/2):
    S = np.linspace(0, s, 20)
    Phi = np.linspace(0, 2*np.pi, 20)
    SS, PPhi = np.meshgrid(S, Phi)
    B = B_norm(S, Phi, Vphi)
    I = trapz(trapz(B*SS, x=S, axis=0), x=Phi, axis=0)
    return I

In [156]:
%%time
Phi = np.linspace(0, 2*np.pi, 100) 
I = current(Vphi=Phi)

CPU times: user 376 ms, sys: 8.18 ms, total: 385 ms
Wall time: 392 ms


In [144]:
fig2, ax2 = plt.subplots(1, 1, figsize=(8,6))

ax2.plot(Phi/(2*np.pi), I)
ax2.grid()

ax2.set_xlabel(r'Time [s]', fontsize=14)
ax2.set_ylabel(r'Induced current $(I_{ind})$ $\left[\frac{4 \pi}{\mu_0 I s^2}\right]$', fontsize=14)
title2 = r'Induced current in the small loop above the current loop rotating at $2 \pi$ rad/s'
ax2.set_title("\n".join(wrap(title2, 60)), fontsize=16)

plt.show()

<Figure size 576x432 with 1 Axes>

In [129]:
x = np.linspace(0, 1, 20)
y = x.copy()

X, Y  = np.meshgrid(x, y)

theta = np.linspace(0, 2*np.pi, 30)

a = np.meshgrid(X, Y, theta)